#### Dot env setup

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

#### Set GEMINI Key

In [ ]:
gemini_api_key = os.getenv('GEMINI_API_KEY')

if gemini_api_key is None:
    print("GEMINI_API_KEY is not set")
    
else:
    os.environ['GEMINI_API_KEY'] = gemini_api_key
    print(f"GEMINI_API_KEY is set")

#### Code to extract meta data from database

##### Database configuration

In [ ]:
import mysql.connector
from mysql.connector import Error


config = {
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'database': os.getenv('DB_NAME')
}

##### Get the tables from the database

In [ ]:
def get_tables():
    try:
        conn = mysql.connector.connect(**config)
        cursor = conn.cursor()
        cursor.execute("SHOW TABLES;")
        tables = [table[0] for table in cursor.fetchall()]
        return tables
    
    except Error as e:
        print(f"Error: {e}")
        return []
    
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()


##### Get Meta data from each table

In [ ]:
def get_metadata(table_name):
    try:
        conn = mysql.connector.connect(**config)
        cursor = conn.cursor()
        query = f"SHOW FULL COLUMNS FROM `{table_name}`"
        cursor.execute(query)
        columns = cursor.fetchall()
        metadata = []
        for column in columns:
            metadata.append(f"Field: {column[0]}, Type: {column[1]}, Collation: {column[2]}, "
                            f"Null: {column[3]}, Key: {column[4]}, Default: {column[5]}, "
                            f"Extra: {column[6]}, Privileges: {column[7]}, Comment: {column[8]}")
        return metadata
    
    except Error as e:
        print(f"Error: {e}")
        return []
    
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()


tables = get_tables()
table_metadata = {}

for table in tables:
    table_metadata[table] = get_metadata(table)

print(table_metadata)

#### Create a document object from the text data

In [ ]:
from llama_index.core import Document
documents = []

for table_name, fields in table_metadata.items():
    table_info = f"Table: {table_name}\n"
    table_info += "\n".join(fields)
    documents.append(Document(text=table_info))

print(f"Number of documents created: {len(documents)}")
for doc in documents:
    print(f"\nDocument content:\n{doc.text}")

#### Setting Embedding Model and Large Language Model

In [ ]:
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding

# Set up Gemini LLM
llm = Gemini(
    model="models/gemini-1.5-flash",
    api_key=gemini_api_key
)


# Set up Gemini Embedding model
embed_model = GeminiEmbedding(
    model_name="models/embedding-001",
    api_key=gemini_api_key)

#### Defining storage context and storing data

In [ ]:
from llama_index.core import ServiceContext, Settings, VectorStoreIndex

# Configure settings
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 1024

# Create service context
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)

#### Creating Index

In [ ]:
# Create index
index = VectorStoreIndex.from_documents(
    documents, 
    service_context=service_context,
    show_progress=True
)

#### Query engine

In [ ]:
query_engine = index.as_query_engine()

#### Generate prompt

In [ ]:
def generate_prompt(user_prompt, metadata):
    prompt = (
        f"Metadata:\n"
        f"{' | '.join(metadata)}\n"
        f"You are given the metadata of the database with the above meta data.\n"
        f"You need to analyze this and write an SQL query in normal text (not even markdown) to answer the below natural language question. "
        f"Don't give any explanation, just write the query.\n"
        f"Question: {user_prompt}"
    )
    return prompt

user_prompt = "Sales people working in tech companies"

#### Generate SQL query by querying from the index

In [ ]:
query = generate_prompt(user_prompt,table_metadata)
response = query_engine.query(query)
print(response)

#### Execute the SQL Query

In [ ]:
def execute_sql_query(query):
    try:
        conn = mysql.connector.connect(**config)
        cursor = conn.cursor(dictionary=True)
        cursor.execute(query)
        results = cursor.fetchall()
        return results
    except Error as e:
        print(f"Error: {e}")
        return None
    finally:
        cursor.close()
        conn.close()

    

results = execute_sql_query(str(response))
print("results\n",results)